# Installs and imports
We do a source install of hyrax and import the major libraries we will need, as well as define constants so all our database accesses are on the same dataset.

In [ ]:
# You must git check out hyrax to ~/rubin-user/hyrax for this to work.
%pip install -q -e ~/rubin-user/hyrax 2>&1 | grep -vE 'WARNING: Error parsing dependencies of (lsst-|astshim|astro-)'
%pip install -q lsdb 2>&1 | grep -vE 'WARNING: Error parsing dependencies of (lsst-|astshim|astro-)'

In [ ]:
import lsdb
import hyrax
from lsst.daf.butler import Butler
import numpy as np
import astropy.units as u

butler_config = {
    "config": "/repo/main",
    "collections": "LSSTComCam/runs/DRP/DP1/v29_0_0_rc5/DM-49865",
}
sky_config = {
    "skymap": "lsst_cells_v1",
    "tract": 5063,
    "patch": 4,
}

# Route 1 -- Create a Hats catalog with objects of interest

In order that this is compatible with DP1 ComCam data, we will pick a tract/patch where there is a deep field, cone search in hats slightly smaller than the patch to avoid edge-of-patch/edge-of-tract cutouts which are not yet handled by LSSTCutout, and then save the catalog file

In [ ]:
lsdb_config = {
    "path": "/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/object_lc",
    "margin_cache": "/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/object_lc_5arcs",
    "columns": [
        "objectId",
        "coord_ra",
        "coord_dec",
        "shape_flag",
        "g_kronMag",
        "g_psfMag",
        "shape_xx",
        "shape_yy",
        "shape_xy",
    ],
}

In [ ]:
# Find the tract/patch dimensions we want
butler = Butler.from_config(**butler_config)
skymap = butler.get("skyMap", sky_config)
tract = skymap[sky_config["tract"]]
patch = tract.getPatchInfo(sky_config["patch"])
wcs = patch.getWcs()
patch_bbox = patch.getInnerBBox()

sky_max = wcs.pixelToSky(patch_bbox.minX, patch_bbox.maxY)
sky_min = wcs.pixelToSky(patch_bbox.maxX, patch_bbox.minY)

ra_range = [sky_min.getLongitude().asDegrees(), sky_max.getLongitude().asDegrees()]
dec_range = [sky_min.getLatitude().asDegrees(), sky_max.getLatitude().asDegrees()]

# Query the catalog and save out the restricted catalog
catalog = lsdb.read_hats(**lsdb_config)
catalog = catalog.box_search(ra=ra_range, dec=dec_range)
catalog = catalog.query("g_psfMag > 20 & g_psfMag < 30")
catalog = catalog.query("shape_flag == False")

catalog._ddf["area_sqpx"] = np.pi * np.sqrt(
    2 * (catalog._ddf["shape_xx"] * catalog._ddf["shape_yy"] - catalog._ddf["shape_xy"] ** 2)
)
catalog = catalog.query("area_sqpx > 5")
res = catalog.compute()
catalog.to_hats(base_catalog_path="./hyrax_catalog", catalog_name="hyrax_catalog", overwrite="True")
# catalog.columns
res

## Setup Hyrax to use the hats catalog
Configure hyrax to use the hats catalog 

In [ ]:
h = hyrax.Hyrax()
h.config["data_set"]["name"] = "LSSTDataset"
h.config["data_set"]["hats_catalog"] = "./hyrax_catalog/"
h.config["data_set"]["butler_repo"] = butler_config["config"]
h.config["data_set"]["butler_collection"] = butler_config["collections"]
h.config["data_set"]["skymap"] = sky_config["skymap"]
h.config["data_set"]["semi_width_deg"] = (20 * u.arcsec).to(u.deg).value
h.config["data_set"]["semi_height_deg"] = (20 * u.arcsec).to(u.deg).value

a = h.prepare()

In [ ]:
a[3].shape

# Route 2 -- Use Butler to Download Catalog 

In [ ]:
butler = Butler(**butler_config)

Let's specify the columns we want to downloda

In [ ]:
INCOLS = [
    "objectId",
    "coord_ra",
    "coord_dec",
]

Now, let's download the data using the butler. We use the `sky_config` we had specified earlier

In [ ]:
object_table = butler.get("object", dataId=sky_config, parameters={"columns": INCOLS})

The butler returns an Astropy table

In [ ]:
object_table

Let's save the first 10,000 images as a pickle file (faster i/o compared to fits)

In [ ]:
import pickle

with open("./test_catalog_10k.pkl", "wb") as f:
    pickle.dump(object_table[:10000], f)

## Setup Hyrax to use the astropy catalog
Configure hyrax to use the astropy catalog. Note that instead of saving the astropy table as a pickle file, you can save it any format that is supported by Astropy

In [ ]:
h = hyrax.Hyrax()
h.config["data_set"]["name"] = "LSSTDataset"
h.config["data_set"]["astropy_table"] = "./test_catalog_10k.pkl"
h.config["data_set"]["butler_repo"] = butler_config["config"]
h.config["data_set"]["butler_collection"] = butler_config["collections"]
h.config["data_set"]["skymap"] = sky_config["skymap"]
h.config["data_set"]["semi_width_deg"] = (20 * u.arcsec).to(u.deg).value
h.config["data_set"]["semi_height_deg"] = (20 * u.arcsec).to(u.deg).value

In [ ]:
a = h.prepare()

In [ ]:
a[1].shape